In [1]:
import sys 
import os

import pymoab
from pymoab import core
from pymoab import types
import numpy as np

In [2]:
pymoab.__class__

module

In [3]:
mb = core.Core()

In [4]:
fs = mb.create_meshset()
mb.load_file("./test_dagmc.h5")
rs = mb.get_root_set()

In [5]:
mb.tag_get_tags_on_entity(rs)

[Name: b'MATERIAL_SET', Type: MB_TYPE_INTEGER, Length: 1.0,
 Name: b'NEUMANN_SET', Type: MB_TYPE_INTEGER, Length: 1.0,
 Name: b'DIRICHLET_SET', Type: MB_TYPE_INTEGER, Length: 1.0,
 Name: b'GEOM_DIMENSION', Type: MB_TYPE_INTEGER, Length: 1.0,
 Name: b'GLOBAL_ID', Type: MB_TYPE_INTEGER, Length: 1.0]

In [6]:
mb.tag_get_handle("NAME", 32, types.MB_TYPE_OPAQUE, False)

Name: b'NAME', Type: MB_TYPE_OPAQUE, Length: 32.0

In [7]:
tg = mb.tag_get_handle("CATEGORY", 32, types.MB_TYPE_OPAQUE, False)
tg

Name: b'CATEGORY', Type: MB_TYPE_OPAQUE, Length: 32.0

In [8]:
x = np.array(["Volume"])


In [9]:
mb.get_entities_by_type_and_tag(rs, types.MB_TYPE_OPAQUE, tg, x)

b'\tempty\n'

In [10]:
mb.get_entities_by_handle(rs,tg)

b''